# TF-IDF値の算出結果比較 (accuracy比較)

## (0) 結果

#### accuracy に大きな変化は見られませんでした。

| テストデータ | サンプル数 | ラベル数 | accuracy（全Bot使用時） | <> | accuracy（単一Bot使用時） |
| :--- | :---: | :---: | :---: | :---: | :---: |
| test_benefitone_conversation | 4,117 | 85 | 0.985436893204 | == | 0.985436893204 |
| test_daikin_conversation | 17,446 | 614 | 0.983723062815 | < | 0.984181568088 |
| test_ptna_conversation | 4,562 | 70 | 0.973707274321 | > | 0.971954425942 |
| test_toyotsu_human_conversation | 5,067 | 302 | 0.996842936069 | < | 0.998421468035 |


## (1) 全Botの学習セットを使用した場合

指定BotのTFIDF値を、全Botの学習セットを使用して算出します。

ベクトライズにはすべてのBotの学習セットを使用しますが、ターゲット「test_toyotsu_human_conversation.csv」に対応するTFIDF値のベクトルだけを取得するようにします。

### (1-1) 環境／テストデータ準備

In [1]:
'''
    テスト環境を準備するためのモジュールを使用します。
'''
import sys
import os
learning_dir = os.path.abspath("../../") #<--- donusagi-bot/learning
os.chdir(learning_dir)

if learning_dir not in sys.path:
    sys.path.append(learning_dir)

from prototype.modules import TestTool

In [2]:
'''
    データファイルは、既存の訓練データを別場所にコピーしてから使用します
    テストデータは、csv_file_name で指定した複数件のファイルを使用します。
'''
csv_file_names = [
    'test_benefitone_conversation.csv',
    'test_daikin_conversation.csv',
    'test_ptna_conversation.csv',
    'test_toyotsu_human_conversation.csv'
]
csv_file_paths = TestTool.copy_testdata_csv(learning_dir, csv_file_names)

CSV file for test=[/Users/makmorit/GitHub/donusagi-bot/learning/prototype/resources/test_benefitone_conversation.csv]
CSV file for test=[/Users/makmorit/GitHub/donusagi-bot/learning/prototype/resources/test_daikin_conversation.csv]
CSV file for test=[/Users/makmorit/GitHub/donusagi-bot/learning/prototype/resources/test_ptna_conversation.csv]
CSV file for test=[/Users/makmorit/GitHub/donusagi-bot/learning/prototype/resources/test_toyotsu_human_conversation.csv]


### (1-2) 検証で使用する関数を準備

In [3]:
import numpy as np
import pandas as pd

from learning.core.training_set.text_array import TextArray

'''
    既存のクラス変数
'''
COUNT_OF_APPEND_BLANK = 3
CLASSIFY_FAILED_ANSWER_ID = 0


def get_all_dataframes_from_csv(csv_file_path):
    '''
        引数のファイルパス配下にある
        全てのCSVファイルパスを取得
    '''
    csv_file_paths = []
    upper_path = os.path.dirname(csv_file_path)
    for f in os.listdir(upper_path):
        if os.path.splitext(f)[1] == '.csv':
            csv_file_paths.append(os.path.join(upper_path, f))
    
    '''
        CSVファイルからpandasデータフレームを生成
        特定Botのデータを抽出できるようにするため、
        データセット名を列として追加
    '''
    dataframes = []
    for path in csv_file_paths:        
        learning_training_messages = pd.read_csv(path, encoding='utf-8')
        learning_training_messages['dataset_name'] = os.path.basename(path)
        dataframes.append(learning_training_messages)        

    return pd.concat(dataframes)


def build_all_training_set_from_csv(csv_file_path):
    '''
        TrainingMessageFromCsv#buildと同様の処理        

        学習セットの全CSVファイルから、
        質問文／ラベル、およびそれらが含まれる
        データセット名の配列を生成
    '''
    learning_training_messages = get_all_dataframes_from_csv(csv_file_path)
    questions = np.array(learning_training_messages['question'])
    answer_ids = np.array(learning_training_messages['answer_id'])
    dataset_names = np.array(learning_training_messages['dataset_name'])

    '''
        空のテキストにラベル0を対応付けるために
        強制的にトレーニングセットを追加
    '''
    questions = np.append(questions, [''] * COUNT_OF_APPEND_BLANK)
    answer_ids = np.append(answer_ids, [CLASSIFY_FAILED_ANSWER_ID] * COUNT_OF_APPEND_BLANK)

    '''
        特定Botのデータを抽出時、ラベル0が含まれるようにするため、
        ラベル0のレコードと、引数のファイル名を関連づけておく
    '''
    target_data_set_name = os.path.basename(csv_file_path)
    dataset_names = np.append(dataset_names, [target_data_set_name] * COUNT_OF_APPEND_BLANK)

    '''
        本来はTrainingMessageFromCsvクラスに内包される変数
        
        body_array: 生成されたボキャブラリを格納
        x, y: 全BotのTF-IDF値が含まれている状態
    '''
    body_array = TextArray(questions) # 
    x_all = body_array.to_vec() 
    y_all = answer_ids
    print('Count all: samples=%d, features=%d, labels=%d' % (
        x_all.shape[0], x_all.shape[1], np.size(np.unique(y_all))
    ))

    '''
        指定Botに関連づけられたデータのインデックスを取得し、
        指定Botに対応するTFIDF値／ラベルだけを抽出する
        --->これが指定Botの学習＋予測に使用されるベクトルx／ラベルyになります
    '''
    target_indices = np.where(dataset_names==target_data_set_name)
    x = x_all[target_indices]
    y = y_all[target_indices]
    print('Count target[%s]: samples=%d, features=%d, labels=%d' % (
        target_data_set_name, x.shape[0], x.shape[1], np.size(np.unique(y))
    ))

    return body_array, x, y

In [4]:
'''
    Bot#__get_estimator から抜粋
'''
from sklearn.grid_search import GridSearchCV
from sklearn.linear_model import LogisticRegression

def get_estimator(training_set_x, training_set_y):
    params = {'C': [10, 100, 140, 200]}
    grid = GridSearchCV(LogisticRegression(), param_grid=params)
    grid.fit(training_set_x, training_set_y)
    estimator = grid.best_estimator_

    return estimator

/Library/Frameworks/Python.framework/Versions/3.5/lib/python3.5/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)
/Library/Frameworks/Python.framework/Versions/3.5/lib/python3.5/site-packages/sklearn/grid_search.py:43: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. This module will be removed in 0.20.
  DeprecationWarning)


### (1-3) 個別のBotについて検証

In [5]:
from learning.core.evaluator import Evaluator

for target_csv_file_path in csv_file_paths:
    '''
        学習実行
    '''
    body_array, x, y = build_all_training_set_from_csv(target_csv_file_path)    
    estimator = get_estimator(x, y)
    
    '''
        accuracy算出
    '''
    evaluator = Evaluator()
    evaluator.evaluate(estimator, x, y, threshold=0.5)

2017/04/26 PM 02:57:32 TextArray#__init__ start
2017/04/26 PM 02:58:00 TextArray#to_vec start
2017/04/26 PM 02:58:00 TextArray#to_vec end


Count all: samples=31183, features=3162, labels=1068
Count target[test_benefitone_conversation.csv]: samples=4117, features=3162, labels=85


2017/04/26 PM 02:58:09 self.threshold: 0.5
2017/04/26 PM 02:58:10 Evaluator#evaluate#elapsed time: 826.369047 ms
2017/04/26 PM 02:58:10 accuracy: 0.985436893204
2017/04/26 PM 02:58:10 TextArray#__init__ start


0.985436893204


2017/04/26 PM 02:58:33 TextArray#to_vec start
2017/04/26 PM 02:58:33 TextArray#to_vec end
/Library/Frameworks/Python.framework/Versions/3.5/lib/python3.5/site-packages/sklearn/cross_validation.py:552: Warning: The least populated class in y has only 2 members, which is too few. The minimum number of labels for any class cannot be less than n_folds=3.
  % (min_labels, self.n_folds)), Warning)


...

0.973707274321


2017/04/26 PM 03:02:51 TextArray#to_vec start
2017/04/26 PM 03:02:51 TextArray#to_vec end


Count all: samples=31183, features=3162, labels=1068
Count target[test_toyotsu_human_conversation.csv]: samples=5067, features=3162, labels=302


2017/04/26 PM 03:03:17 self.threshold: 0.5
2017/04/26 PM 03:03:21 Evaluator#evaluate#elapsed time: 3832.440138 ms
2017/04/26 PM 03:03:21 accuracy: 0.996842936069


0.996842936069


## (2) 単一Botの学習セットだけを使用した場合

従来の方法により実行します。

In [6]:
'''
    Bot#learn と同様の処理
'''
from learning.core.training_set.training_message_from_csv import TrainingMessageFromCsv

for target_csv_file_path in csv_file_paths:
    '''
        学習セットをCSVファイルから個別に取得し、
        学習実行
    '''
    training_set = TrainingMessageFromCsv(None, target_csv_file_path, None)
    training_set.build()
    print('Count target[%s]: samples=%d, features=%d, labels=%d' % (
        os.path.basename(target_csv_file_path), 
        training_set.x.shape[0], 
        training_set.x.shape[1], 
        np.size(np.unique(training_set.y))
    ))
    estimator = get_estimator(training_set.x, training_set.y)

    '''
        accuracy算出
    '''
    evaluator = Evaluator()
    evaluator.evaluate(estimator, training_set.x, training_set.y, threshold=0.5)

2017/04/26 PM 03:03:21 TrainingMessageFromCsv#__build_learning_training_messages count of learning data: 4114
2017/04/26 PM 03:03:21 TextArray#__init__ start
2017/04/26 PM 03:03:23 TextArray#to_vec start
2017/04/26 PM 03:03:23 TextArray#to_vec end


Count target[test_benefitone_conversation.csv]: samples=4117, features=644, labels=85


2017/04/26 PM 03:03:27 self.threshold: 0.5
2017/04/26 PM 03:03:28 Evaluator#evaluate#elapsed time: 414.023161 ms
2017/04/26 PM 03:03:28 accuracy: 0.985436893204
2017/04/26 PM 03:03:28 TrainingMessageFromCsv#__build_learning_training_messages count of learning data: 17443
2017/04/26 PM 03:03:28 TextArray#__init__ start


0.985436893204


2017/04/26 PM 03:03:38 TextArray#to_vec start
2017/04/26 PM 03:03:38 TextArray#to_vec end
/Library/Frameworks/Python.framework/Versions/3.5/lib/python3.5/site-packages/sklearn/cross_validation.py:552: Warning: The least populated class in y has only 2 members, which is too few. The minimum number of labels for any class cannot be less than n_folds=3.
  % (min_labels, self.n_folds)), Warning)


...

0.971954425942


2017/04/26 PM 03:07:26 TextArray#to_vec start
2017/04/26 PM 03:07:26 TextArray#to_vec end


Count target[test_toyotsu_human_conversation.csv]: samples=5067, features=1182, labels=302


2017/04/26 PM 03:07:51 self.threshold: 0.5
2017/04/26 PM 03:07:54 Evaluator#evaluate#elapsed time: 2945.917130 ms
2017/04/26 PM 03:07:54 accuracy: 0.998421468035


0.998421468035
